# Land Use Classification using TensorFlow from scratch
In this notebook, we illustrate how one can produce a deep learning model to classify aerial images based on land use type (developed, forested, cultivated, etc.). We apply transfer learning with MicrosoftML to adapt a pretrained featurizer for our classification use case.

<a name="prep"></a>
## Prepare deep learning framework-specific input files

If you have not generated your own training and validation sets through image extraction, download the following files and decompress them in your VM's temporary (`D:\`) storage:
- [Balanced training image set (~3 GB)](https://mawahstorage.blob.core.windows.net/aerialimageclassification/imagesets/balanced_training_set.zip)
- [Balanced validation image set (~1 GB)](https://mawahstorage.blob.core.windows.net/aerialimageclassification/imagesets/balanced_validation_set.zip)

The image sets linked above contain raw PNG images sorted into folders by their assigned label. 

Update the `training_image_dir` variable below to reflect the directory where your training and validation sets have been saved. The `label_to_number_dict` variable specifies the correspondence between the label names and a numeric code; it does not need to be modified unless you have changed the labeling scheme.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import os, sys
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.decomposition import IncrementalPCA
from sklearn.metrics import confusion_matrix, accuracy_score
from scipy.ndimage import imread

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn

import utils.connection_settings as cs
import utils.land_use_classification_utils as luc

training_image_dir = cs.TRAINING_IMAGE_DIR
validation_image_dir = cs.VALIDATION_IMAGE_DIR
test_image_dir = cs.TEST_IMAGE_DIR
label_to_number_dict = {'Barren': 0,
                        'Forest': 1,
                        'Shrub': 2,
                        'Cultivated': 3,
                        'Herbaceous': 4,
                        'Developed': 5}

tf.logging.set_verbosity(tf.logging.INFO)

# Autoreload when modules are changed
%load_ext autoreload
%autoreload 2

# Show matplotlib plots inline
%matplotlib inline

In [2]:
# Todo: make model smaller
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    nn = tf.reshape(features, [-1, 224, 224, 3])

    # Convolutional Layer #1
    nn = tf.layers.conv2d(
        inputs=nn,
        filters=32,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu
    )

    # Pooling Layer #1
    nn = tf.layers.max_pooling2d(inputs=nn, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    nn = tf.layers.conv2d(
        inputs=nn,
        filters=64,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu
    )
    nn = tf.layers.max_pooling2d(inputs=nn, pool_size=[2, 2], strides=2)
    
    # Convolutional Layer #3 and Pooling Layer #3
    nn = tf.layers.conv2d(
        inputs=nn,
        filters=64,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu
    )
    nn = tf.layers.max_pooling2d(inputs=nn, pool_size=[2, 2], strides=2)

    # Dense Layer
    nn = tf.contrib.layers.flatten(nn)
    nn = tf.layers.dense(inputs=nn, units=512, activation=tf.nn.relu)
    nn = tf.layers.dropout(inputs=nn, rate=0.4, training=mode == learn.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=nn, units=10)

    loss = None
    train_op = None

    # Calculate Loss (for both TRAIN and EVAL modes)
    if mode != learn.ModeKeys.INFER:
        onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
        loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=1e-4,
            optimizer='Adam'
        )

    # Generate Predictions
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    # Return a ModelFnOps object
    return model_fn.ModelFnOps(mode=mode, predictions=predictions, loss=loss, train_op=train_op)

In [3]:
if 'train_data' in locals():
    print("Freeing up memory...")
    del train_data
    
if 'eval_data' in locals():
    print("Freeing up memory...")
    del eval_data

print("Load training data")
train_info = luc.gather_data(training_image_dir, label_to_number_dict)

train_data = np.empty((train_info.shape[0],224,224,3), np.float32)
for index, row in train_info.iterrows():
    path = row.iloc[0]
    image = imread(path)
    image = np.expand_dims(image, axis=0)
    train_data[index, :] = np.array(image)

train_labels = train_info[train_info.columns[1]].values

print(train_data.shape)
print(train_labels.shape)

print("Load eval data")
eval_info = luc.gather_data(validation_image_dir, label_to_number_dict)

eval_data = np.empty((eval_info.shape[0],224,224,3), np.float32)
for index, row in eval_info.iterrows():
    path = row.iloc[0]
    image = imread(path)
    image = np.expand_dims(image, axis=0)
    eval_data[index, :] = np.array(image)

eval_labels = eval_info[eval_info.columns[1]].values

print(eval_data.shape)
print(eval_labels.shape)

Load training data
(44184, 224, 224, 3)
(44184,)
Load eval data
(5880, 224, 224, 3)
(5880,)


In [4]:
# Create the Estimator
classifier = learn.SKCompat(learn.Estimator(model_fn=cnn_model_fn, model_dir="/tmp/convnet_model"))

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_master': '', '_save_checkpoints_steps': None, '_model_dir': '/tmp/convnet_model', '_is_chief': True, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_num_worker_replicas': 0, '_environment': 'local', '_task_type': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000019519D5B240>, '_session_config': None, '_task_id': 0, '_num_ps_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_evaluation_master': ''}


INFO:tensorflow:Using config: {'_master': '', '_save_checkpoints_steps': None, '_model_dir': '/tmp/convnet_model', '_is_chief': True, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_num_worker_replicas': 0, '_environment': 'local', '_task_type': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000019519D5B240>, '_session_config': None, '_task_id': 0, '_num_ps_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_evaluation_master': ''}


In [5]:
# Train the model
classifier.fit(
    x=train_data,
    y=train_labels,
    batch_size=100,
    steps=5000
)

INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Saving checkpoints for 1 into /tmp/convnet_model\model.ckpt.


INFO:tensorflow:Saving checkpoints for 1 into /tmp/convnet_model\model.ckpt.


INFO:tensorflow:loss = 25.387, step = 1


...

INFO:tensorflow:Saving checkpoints for 5000 into /tmp/convnet_model\model.ckpt.


INFO:tensorflow:Saving checkpoints for 5000 into /tmp/convnet_model\model.ckpt.


INFO:tensorflow:Loss for final step: 0.137604.


INFO:tensorflow:Loss for final step: 0.137604.


SKCompat()

In [6]:
# Configure the accuracy metric for evaluation
metrics = {
    "accuracy": learn.MetricSpec(metric_fn=tf.metrics.accuracy, prediction_key="classes"),
}

In [7]:
# Evaluate the model and print results
eval_results = classifier.score(
    x=eval_data,
    y=eval_labels,
    metrics=metrics
)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2017-08-17-17:59:12


INFO:tensorflow:Starting evaluation at 2017-08-17-17:59:12


INFO:tensorflow:Restoring parameters from /tmp/convnet_model\model.ckpt-5000


INFO:tensorflow:Restoring parameters from /tmp/convnet_model\model.ckpt-5000


INFO:tensorflow:Finished evaluation at 2017-08-17-18:05:25


INFO:tensorflow:Finished evaluation at 2017-08-17-18:05:25


INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.732483, global_step = 5000, loss = 1.12997


INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.732483, global_step = 5000, loss = 1.12997


{'accuracy': 0.73248297, 'loss': 1.1299675, 'global_step': 5000}
